# PREVISÃO DOS PREÇOS

### Respondendo a pergunta 3 da seção Entregas

Para a previsão do preço a partir dos dados, podemos abordar o problema como uma REGRESSÃO, onde nosso objetivo é estimar o valor contínuo do preço dos carros com base nas variáveis disponíveis.

Como não há regras reais de negócios,tanto quanto interesses específicos para este projeto, realizei uma pesquisa breve e considerei com nos dados, estas colunas como as mais relevantes, sendo assim , podemos utilizar as seguintes variáveis como entrada para o modelo de regressão:

1. Marca: Transformar a coluna de marca em variáveis binárias (dummy variables) para capturar a presença ou ausência de cada marca no conjunto de dados. Isso permitirá que o modelo leve em consideração o efeito da marca no preço.

2. Modelo: Da mesma forma, podemos transformar a coluna de modelo em variáveis binárias para cada modelo presente nos dados.

3. Ano de fabricação: Utilizar o ano de fabricação diretamente como uma variável numérica.

4. Hodômetro: Manter o hodômetro como uma variável numérica.

5. Tipo de vendedor: Transformar a coluna de tipo de vendedor em variáveis binárias para capturar a presença ou ausência de cada tipo de vendedor.

6. Tipo de câmbio: Transformar a coluna de tipo de câmbio em variáveis binárias para cada tipo presente nos dados.

7. Blindado: Transformar a coluna de blindado em variáveis binárias para capturar a presença ou ausência de carros blindados.

8. Estado: Transformar a coluna de estado em variáveis binárias para cada estado presente nos dados.

9. Preço: Manter o preco como uma variável numérica.          

Com relação ao modelo, uma abordagem comum para previsão de preços de carros é a utilização de regressão linear.

Para avaliar o desempenho do modelo de regressão, foram utilizadas duas métricas comuns: R² (coeficiente de determinação) e MAE (erro absoluto médio).

O coeficiente de determinação (R²) mede a proporção da variabilidade da variável de resposta (preço dos carros) que é explicada pelo modelo. Ele varia de 0 a 1, onde 0 indica que o modelo não é capaz de explicar a variabilidade dos dados e 1 indica que o modelo explica perfeitamente a variabilidade.

O erro absoluto médio (MAE) é uma medida da diferença média entre os valores previstos e os valores reais do preço dos carros. Ele representa a magnitude média dos erros de previsão e fornece uma ideia do quão bem o modelo está fazendo a previsão em termos de unidades da variável de resposta.

Essas métricas nos ajudam a avaliar o desempenho do modelo de regressão e fornecem uma medida quantitativa de quão bem o modelo está fazendo a previsão dos preços dos carros. Um valor mais alto de R² e um valor mais baixo de MAE indicam um melhor desempenho do modelo.

### Importando as bibliotecas uteis

In [109]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt

In [110]:
df_cars_train = pd.read_csv("df_cars_train_final.csv")

In [111]:
# Selecionar as colunas relevantes
colunas_relevantes = ['marca', 'modelo', 'ano_de_fabricacao', 'hodometro', 'cambio', 'tipo_vendedor', 'blindado', 'estado_vendedor', 'preco']
df_relevante = df_cars_train[colunas_relevantes]

# Aplicar a transformação logarítmica
df_relevante['preco'] = np.log1p(df_relevante['preco'])

# Exibir o novo DataFrame com as colunas relevantes
df_relevante.head()

,marca,modelo,ano_de_fabricacao,hodometro,cambio,tipo_vendedor,blindado,estado_vendedor,preco
0,NISSAN,KICKS,2017,67772.0,CVT,PF,N,São Paulo (SP),11.221685
1,JEEP,COMPASS,2017,62979.0,Automática,PF,N,Minas Gerais (MG),11.314064
2,KIA,SORENTO,2018,44070.0,Automática,PJ,N,São Paulo (SP),12.000436
3,VOLKSWAGEN,AMAROK,2013,85357.0,Automática,PJ,N,São Paulo (SP),11.725472
4,SSANGYONG,KORANDO,2013,71491.0,Automática,PF,N,Rio de Janeiro (RJ),11.319593


### Transformando as colunas categóricas em variáveis binárias (dummy variables)

In [112]:
# Selecionar todas as colunas não numéricas
colunas_categoricas = df_relevante.select_dtypes(include=['object', 'bool']).columns

# Transformar as colunas não numéricas em variáveis binárias
df_binario = pd.get_dummies(df_relevante, columns=colunas_categoricas, drop_first=True)

# Exibir df_binarioo novo dataframe com as variáveis binárias
df_binario.head()

,ano_de_fabricacao,hodometro,preco,marca_AUDI,marca_BMW,marca_BRM,marca_CHERY,marca_CHEVROLET,marca_CHRYSLER,marca_CITROËN,...,estado_vendedor_Piauí (PI),estado_vendedor_Rio Grande do Norte (RN),estado_vendedor_Rio Grande do Sul (RS),estado_vendedor_Rio de Janeiro (RJ),estado_vendedor_Rondônia (RO),estado_vendedor_Roraima (RR),estado_vendedor_Santa Catarina (SC),estado_vendedor_Sergipe (SE),estado_vendedor_São Paulo (SP),estado_vendedor_Tocantins (TO)
0,2017,67772.0,11.221685,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2017,62979.0,11.314064,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2018,44070.0,12.000436,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2013,85357.0,11.725472,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2013,71491.0,11.319593,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Divisão dos dados

In [113]:
X = df_binario.drop('preco', axis=1)
y = df_binario['preco']

### Divisão em conjunto de treinamento e teste

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Criação do modelo de previsão de preços

In [115]:
model = XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

### Avaliação do modelo

In [116]:
y_pred = model.predict(X_test)

from sklearn.metrics import r2_score, mean_absolute_error

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('R²:', r2)
print('MAE:', mae)

R²: 0.6908761456110516
MAE: 0.24092141869048744


Um R² de 0.6909 indica que o modelo tem uma capacidade razoável de explicar a variabilidade dos preços com base nas variáveis independentes utilizadas. 

No entanto, o MAE de 0.2409 indica que ainda existe uma margem de erro considerável nas previsões de preços realizadas pelo modelo.

### Obtendo os valores previstos

In [117]:
print(y_pred)

[11.746242  11.2765255 11.744368  ... 10.9049635 12.511687  12.481439 ]


### Comparando os valores originais com os valores previstos

In [118]:
# Converter os valores previstos em um DataFrame
df_pred = pd.DataFrame({'Preço Original': y_test, 'Preço Previsto': y_pred})

# Imprimir o DataFrame
print(df_pred)

       Preço Original  Preço Previsto
7936        12.081927       11.746242
22472       11.064890       11.276525
18848       11.613554       11.744368
24252       12.941950       12.511095
17123       11.680345       11.594167
...               ...             ...
14865       11.667642       11.776496
13630       11.779896       11.283760
3509        10.681556       10.904963
27544       12.597950       12.511687
11176       12.456685       12.481439

[5917 rows x 2 columns]


In [119]:
# Aplicar a transformação inversa logarítmica nos valores das colunas "Preço Original" e "Preço Previsto"
df_pred['Preço Original'] = np.expm1(df_pred['Preço Original'])
df_pred['Preço Previsto'] = np.expm1(df_pred['Preço Previsto'])

# Imprimir o DataFrame com os valores desfeitos da transformação logarítmica
print(df_pred)

       Preço Original  Preço Previsto
7936    176649.291489   126277.062500
22472    63887.181849    78945.484375
18848   110585.626201   126040.640625
24252   417461.596397   271330.093750
17123   118224.015684   108462.257812
...               ...             ...
14865   116731.681919   130155.898438
13630   130599.138413    79518.703125
3509     43544.236441    54444.937500
27544   295950.256041   271490.812500
11176   256961.328558   263401.531250

[5917 rows x 2 columns]


Ao analisar os resultados, podemos ver que, em alguns casos, os valores previstos pelo modelo se aproximam dos valores reais ("Preço Original"), enquanto em outros casos há uma diferença considerável entre os valores previstos e os valores reais.

É importante considerar que a qualidade da previsão do modelo pode variar de acordo com a complexidade dos dados e a capacidade do modelo em capturar os padrões subjacentes.

## Vamos aplicar a previsão ao dataframe df_cars_test_final

In [120]:
df_cars_test = pd.read_csv("df_cars_test_final.csv")

### Transformando as colunas categóricas em variáveis binárias (dummy variables)

In [121]:
# Selecionar todas as colunas não numéricas
colunas_categoricas_2 = df_cars_test.select_dtypes(include=['object', 'bool']).columns

# Transformar as colunas não numéricas em variáveis binárias
df_binario_2 = pd.get_dummies(df_relevante, columns=colunas_categoricas, drop_first=True)

# Exibir df_binarioo novo dataframe com as variáveis binárias
df_binario_2.head()

,ano_de_fabricacao,hodometro,preco,marca_AUDI,marca_BMW,marca_BRM,marca_CHERY,marca_CHEVROLET,marca_CHRYSLER,marca_CITROËN,...,estado_vendedor_Piauí (PI),estado_vendedor_Rio Grande do Norte (RN),estado_vendedor_Rio Grande do Sul (RS),estado_vendedor_Rio de Janeiro (RJ),estado_vendedor_Rondônia (RO),estado_vendedor_Roraima (RR),estado_vendedor_Santa Catarina (SC),estado_vendedor_Sergipe (SE),estado_vendedor_São Paulo (SP),estado_vendedor_Tocantins (TO)
0,2017,67772.0,11.221685,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2017,62979.0,11.314064,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2018,44070.0,12.000436,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2013,85357.0,11.725472,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2013,71491.0,11.319593,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Divisão dos dados

In [122]:
X_2 = df_binario_2
y_2 = df_binario['preco']

### Divisão em conjunto de treinamento e teste

In [123]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

### Criação do modelo de previsão de preços

In [124]:
model_2 = XGBRegressor()
model_2.fit(X_train_2, y_train_2)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

### Avaliação do modelo

In [125]:
y_pred_2 = model_2.predict(X_test_2)


r2_2 = r2_score(y_test, y_pred)
mae_2 = mean_absolute_error(y_test, y_pred)

print('R²_2:', r2_2)
print('MAE_2:', mae_2)

R²_2: 0.6908761456110516
MAE_2: 0.24092141869048744


Os resultados são exatamente iguais ao conjuntpo de dados de treino

### Obtendo os valores previstos

In [153]:
print(y_pred_2)

[12.081365 11.066562 11.612119 ... 10.682891 12.599522 12.458372]


### Comparando os valores Originais com os valores previstos

In [154]:
# Converter os valores previstos em um DataFrame
df_pred_2 = pd.DataFrame({'Preço Original': y_test, 'Preço Previsto_2': y_pred_2})

# Imprimir o DataFrame
print(df_pred_2)

       Preço Original  Preço Previsto_2
7936        12.081927         12.081365
22472       11.064890         11.066562
18848       11.613554         11.612119
24252       12.941950         12.939278
17123       11.680345         11.678919
...               ...               ...
14865       11.667642         11.663966
13630       11.779896         11.778339
3509        10.681556         10.682891
27544       12.597950         12.599522
11176       12.456685         12.458372

[5917 rows x 2 columns]


In [155]:
# Aplicar a transformação inversa logarítmica nos valores das colunas "Preço Original" e "Preço Previsto"
df_pred_2['Preço Original'] = np.expm1(df_pred_2['Preço Original'])
df_pred_2['Preço Previsto_2'] = np.expm1(df_pred_2['Preço Previsto_2'])

# Imprimir o DataFrame com os valores desfeitos da transformação logarítmica
print(df_pred_2)

       Preço Original  Preço Previsto_2
7936    176649.291489     176549.921875
22472    63887.181849      63994.093750
18848   110585.626201     110426.968750
24252   417461.596397     416347.375000
17123   118224.015684     118055.531250
...               ...               ...
14865   116731.681919     116303.398438
13630   130599.138413     130396.062500
3509     43544.236441      43602.421875
27544   295950.256041     296415.750000
11176   256961.328558     257395.281250

[5917 rows x 2 columns]


Ao analisar os resultados da comparação entre os valores originais e os valores previstos, podemos observar que há uma proximidade entre eles. 

Em geral, os valores previstos estão muito próximos dos valores originais, o que indica que o modelo tem uma boa capacidade de fazer previsões precisas. No entanto, é possível notar que em alguns casos há uma pequena diferença entre os valores previstos e os valores originais, indicando que o modelo não consegue prever com 100% de precisão todos os casos.

Essas diferenças podem ser resultado de diversas variáveis que podem afetar o preço dos carros e que não foram consideradas no modelo. Além disso, podem existir outros fatores não capturados pelo modelo que influenciam o preço dos carros.

### Salvando o arquivo final

In [156]:
# Criar DataFrame com as colunas id e preço previsto
df_result = pd.DataFrame({'id': df_cars_test['id'], 'preco': df_pred_2['Preço Previsto_2']})

# Salvar o DataFrame em um arquivo CSV
df_result.to_csv('predicted.csv', index=False)